# CS 440 Project on Rubik's Cube solving with Reinforcement Learning
## 3x3 Version

_by David Edwards, December 12, 2017_

## Introduction

I was fairly happy with my [2x2](Edwards-Project.ipynb) Rubik's Cube solution, but after discussing all of changes that would need to be made for the 3x3 solution, I decided to go ahead and make those changes.  Below find the code with some discussion.  So, let's do this:  <img src="https://edwards.cloud/wp-content/uploads/2017/12/3x3.jpg">

## Methods

I needed to make changes to the following functions to get everything working with a 3x3:


In [1]:
import copy as copy
import numpy as np
import random
import time

FRONT = 1
LEFT = 0
TOP = 2
BOTTOM = 3
RIGHT = 4
BACK = 5

def printFirstLetters3x3(pair):
    return("{}\t{}\t{}\t".format(pair[0][:1],pair[1][:1],pair[2][:1]))


def printState3x3(state):
    '''
    Prints a Tower of Hanoi state.  Now, with added pegs.
    :param state: list of lists representing tower of hanoi state
    :return: prints out the state nicely.
    '''
    print("\t\t\t", printFirstLetters3x3(state[TOP][0]))
    print("\t\t\t", printFirstLetters3x3(state[TOP][1]))
    print("\t\t\t", printFirstLetters3x3(state[TOP][2]))
    print(printFirstLetters3x3(state[LEFT][0]), printFirstLetters3x3(state[FRONT][0]),printFirstLetters3x3(state[RIGHT][0]),printFirstLetters3x3(state[BACK][0]))
    print(printFirstLetters3x3(state[LEFT][1]), printFirstLetters3x3(state[FRONT][1]),printFirstLetters3x3(state[RIGHT][1]),printFirstLetters3x3(state[BACK][1]))
    print(printFirstLetters3x3(state[LEFT][2]), printFirstLetters3x3(state[FRONT][2]),printFirstLetters3x3(state[RIGHT][2]),printFirstLetters3x3(state[BACK][2]))
    print("\t\t\t", printFirstLetters3x3(state[BOTTOM][0]))
    print("\t\t\t", printFirstLetters3x3(state[BOTTOM][1]))
    print("\t\t\t", printFirstLetters3x3(state[BOTTOM][2]))
    print("-------------------------------")
    

def makeMove3x3(state, move, printMoves=False):
    '''
    Takes a move and makes it 2x2 rubik's cube
    :param state: list of lists representing cube
    :param move: possible rubik's cube move
    :return:the state after the move was made
    '''

    if move == "U":
        state[LEFT][0], state[FRONT][0], state[RIGHT][0], state[BACK][0] = state[FRONT][0], state[RIGHT][0], state[BACK][0], state[LEFT][0]
    if move == "Uprime":
        state[FRONT][0], state[RIGHT][0], state[BACK][0], state[LEFT][0] = state[LEFT][0], state[FRONT][0], state[RIGHT][0], state[BACK][0]
    if move == "D":
        state[LEFT][2], state[FRONT][2], state[RIGHT][2], state[BACK][2] = state[FRONT][2], state[RIGHT][2], state[BACK][2], state[LEFT][2]
    if move == "Dprime":
        state[FRONT][2], state[RIGHT][2], state[BACK][2], state[LEFT][2] = state[LEFT][2], state[FRONT][2], state[RIGHT][2], state[BACK][2]
    if move == "R":
        state[TOP][0][2], state[TOP][1][2], state[TOP][2][2], state[FRONT][0][2], state[FRONT][1][2], state[FRONT][2][2], state[BOTTOM][0][2], state[BOTTOM][1][2], state[BOTTOM][2][2], state[BACK][0][2], state[BACK][1][2], state[BACK][2][2] = state[FRONT][0][2], state[FRONT][1][2], state[FRONT][2][2], state[BOTTOM][0][2], state[BOTTOM][1][2], state[BOTTOM][2][2], state[BACK][0][2], state[BACK][1][2], state[BACK][2][2],state[TOP][0][2], state[TOP][1][2], state[TOP][2][2]
    if move == "Rprime":
        state[FRONT][0][2], state[FRONT][1][2], state[FRONT][2][2], state[BOTTOM][0][2], state[BOTTOM][1][2], state[BOTTOM][2][2], state[BACK][0][2], state[BACK][1][2], state[BACK][2][2], state[TOP][0][2], state[TOP][1][2], state[TOP][2][2] = state[TOP][0][2], state[TOP][1][2], state[TOP][2][2], state[FRONT][0][2], state[FRONT][1][2], state[FRONT][2][2], state[BOTTOM][0][2], state[BOTTOM][1][2], state[BOTTOM][2][2], state[BACK][0][2], state[BACK][1][2], state[BACK][2][2]
    if move == "L":
        state[TOP][0][0], state[TOP][1][0], state[TOP][2][0], state[FRONT][0][0], state[FRONT][1][0], state[FRONT][2][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BOTTOM][2][0], state[BACK][0][0], state[BACK][1][0], state[BACK][2][0] = state[FRONT][0][0], state[FRONT][1][0], state[FRONT][2][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BOTTOM][2][0], state[BACK][0][0], state[BACK][1][0], state[BACK][2][0],state[TOP][0][0], state[TOP][1][0], state[TOP][2][0]
    if move == "Lprime":
        state[FRONT][0][0], state[FRONT][1][0], state[FRONT][2][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BOTTOM][2][0], state[BACK][0][0], state[BACK][1][0], state[BACK][2][0], state[TOP][0][0], state[TOP][1][0], state[TOP][2][0] = state[TOP][0][0], state[TOP][1][0], state[TOP][2][0], state[FRONT][0][0], state[FRONT][1][0], state[FRONT][2][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BOTTOM][2][0], state[BACK][0][0], state[BACK][1][0], state[BACK][2][0]

    if printMoves:
        printState3x3(state)

    return state


def winner3x3(state):
        '''
        Determines if a winning state occured
        :param state: list of lists representing tower of hanoi state
        :return: True if winning state, False otherwise.
        '''

        return faceComplete3x3(state[LEFT]) and faceComplete3x3(state[FRONT]) and faceComplete3x3(state[TOP]) and faceComplete3x3(
            state[BOTTOM]) and faceComplete3x3(state[RIGHT]) and faceComplete3x3(state[BACK])


def faceComplete3x3(face):
    return (face[0][0]== face[0][1] == face[0][2] == face[1][0] == face[1][1] == face[1][2] == face[2][0] == face[2][1] == face[2][2])


def epsilonGreedy(epsilon, Q, state, validMovesF,tupleF):
    '''
    Makes either a random move, or tries the move which Q indicates is the best.
    :param epsilon: A decreasing number representing the level of randomness
    :param Q: Dictionary of state,move - value pairs, with the higher values being better moves
    :param state: list of lists representing tower of hanoi state
    :param validMovesF: function returning valid moves
    :return:
    '''
    goodMoves = validMovesF(state)
    if np.random.uniform() < epsilon:
        # Random Move
        return random.choice(goodMoves)
    else:
        # Greedy Move
        Qs = np.array([Q.get(tupleF(state,m), 0.0) for m in goodMoves])
        return goodMoves[np.argmax(Qs)]
    

def trainQ(startState, nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF, winnerF, tupleF):
    '''
    Creates and fills a dictionary, Q, representing the (state,move) - value pairs which, if followed
    should create the shortest path to the solution.
    :param nRepetitions: how many times to iterate through.  Higher numbers would generate more accurate results
    :param learningRate: How much to adjust the value part of the dictionary
    :param epsilonDecayFactor: how quickly to reduce the random factor.
    :param validMovesF: function returning valid moves of a state
    :param makeMoveF: function making a move on a state
    :return: the dictionary, Q, and a list containing the number of steps it took per iteration to find the goal state
    '''
    maxGames = nRepetitions
    rho = learningRate
    epsilonDecayRate = epsilonDecayFactor
    epsilon = 1.0
    Q = {}
    stepList = []
    # show the moves while debuggin
    showMoves = False

    for nGames in range(maxGames):
        # if nGames % 10 == 0: print(".", end="")
        # if nGames % 100 == 0: print("Q length: ", len(Q))
        # reduce the randomness every pass
        epsilon *= epsilonDecayRate
        step = 0
        # hardcoded start state
        state = startState
        done = False

        while not done:
            #if step % 100 == 0: print(".", end="")
            #if step % 1000 == 0: print("Q length: ", len(Q))
            step += 1
            # grab either a random or best of the known moves
            move = epsilonGreedy(epsilon, Q, state, validMovesF,tupleF)

            # we don't want to change state directly, and because state is a list of lists, need to do a
            # deepcopy on it, then make the move
            stateNew = copy.deepcopy(state)
            makeMoveF(stateNew, move)

            # if we haven't encountered this state,move combo, add it to Q
            if tupleF(state, move) not in Q:
                Q[tupleF(state, move)] = 0.0  # Initial Q value for new state, move

            # print if debugging
            if showMoves:
                printState3x3(stateNew)
            if winnerF(stateNew):
                # We won!  backfill Q
                # print('End State, we won!')
                Q[tupleF(state, move)] = -1.0
                done = True
                # we're keeping a list of the number of steps it took for each winning solution, so add it here.
                stepList.append(step)

            # update the Q which led us here using the learning factor, and the difference between the current state
            # and the old state
            if step > 1:
                Q[tupleF(stateOld, moveOld)] += rho * (-1 + Q[tupleF(state, move)] - Q[tupleF(stateOld, moveOld)])
            # Store the current state, move so we can access it for the next Q update
            stateOld, moveOld = state, move
            state = stateNew

    return Q, stepList


def testQ(Q, initialState, maxSteps, validMovesF, makeMoveF, winnerF, tupleF):
    '''
    Using the dictionary Q, and the initial state of the game, traverse and return the best path.
    :param Q: dictionary representing the (state,move) - value pairs which, if followed should create the shortest path to the solution.
    :param maxSteps: The number of steps to attempt before giving up.
    :param validMovesF: function returning valid moves of a state
    :param makeMoveF: function making a move on a state
    :return: list containing the states from start to finish
    '''
    state = initialState

    statePath = []
    movePath = []
    movePath.append("Initial")
    statePath.append(state)

    for i in range(maxSteps):
        if winnerF(state):
            return statePath, movePath
        goodMoves = validMovesF(state)
        Qs = np.array([Q.get(tupleF(state, m), -1000.0) for m in goodMoves])
        move = goodMoves[np.argmax(Qs)]
        movePath.append(move)
        nextState = copy.deepcopy(state)
        makeMoveF(nextState, move)
        statePath.append(nextState)
        state = nextState

    return "No path found",movePath


def validMoves(state):
    '''
    Returns a list of lists representing valid rubik's cube moves from the given state.
    Move Rotations courtesy: http://www.rubiksplace.com/move-notations/
    :param state: list of lists representing tower of hanoi state
    :return: list of lists representing valid tower of hanoi moves from the given state.
    '''

    validStates = ["U", "Uprime", "D", "Dprime", "R", "Rprime", "L", "Lprime"]

    return validStates


def getTuple3x3(state, move):
    '''
    Need immutable type for key to dictionary
    :param state: list of lists representing tower of hanoi state
    :return: tuple representation of the state
    '''
    superTuple = tuple(tuple(tuple(s[0])+tuple(s[1])+tuple(s[2])) for s in state)
    return (superTuple, move)


Basically, all of the changes involved having a 3x3 array everywhere there was formerly a 2x2.  `validMoves` didn't change, as it's generally hard for a human to just rotate the middle row or column.  Typically that's a R,L, or an U,D move.  

I added a `winnerF` and `tupleF` function to the parameters for testQ and trainQ, as it was using a hard-coded winner  and getTuple function previously.  

So, let's try it out!

In [2]:
completeState = [[["Red", "Red", "Red"],["Red","Red", "Red"],["Red","Red", "Red"]],[["Blue", "Blue","Blue"],["Blue", "Blue","Blue"],["Blue", "Blue","Blue"]],[["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"]],[["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"]],[["White", "White", "White"],["White", "White", "White"],["White", "White", "White"]],[["Green", "Green", "Green"],["Green", "Green", "Green"],["Green", "Green", "Green"]]]
completeState

[[['Red', 'Red', 'Red'], ['Red', 'Red', 'Red'], ['Red', 'Red', 'Red']],
 [['Blue', 'Blue', 'Blue'],
  ['Blue', 'Blue', 'Blue'],
  ['Blue', 'Blue', 'Blue']],
 [['Yellow', 'Yellow', 'Yellow'],
  ['Yellow', 'Yellow', 'Yellow'],
  ['Yellow', 'Yellow', 'Yellow']],
 [['Orange', 'Orange', 'Orange'],
  ['Orange', 'Orange', 'Orange'],
  ['Orange', 'Orange', 'Orange']],
 [['White', 'White', 'White'],
  ['White', 'White', 'White'],
  ['White', 'White', 'White']],
 [['Green', 'Green', 'Green'],
  ['Green', 'Green', 'Green'],
  ['Green', 'Green', 'Green']]]

Now, let's see how that displays:

In [3]:
printState3x3(completeState)

			 Y	Y	Y	
			 Y	Y	Y	
			 Y	Y	Y	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
			 O	O	O	
			 O	O	O	
			 O	O	O	
-------------------------------


Looks like a 3x3 cube to me.

In [4]:
completeState = [[["Red", "Red", "Red"],["Red","Red", "Red"],["Red","Red", "Red"]],[["Blue", "Blue","Blue"],["Blue", "Blue","Blue"],["Blue", "Blue","Blue"]],[["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"]],[["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"]],[["White", "White", "White"],["White", "White", "White"],["White", "White", "White"]],[["Green", "Green", "Green"],["Green", "Green", "Green"],["Green", "Green", "Green"]]]
print("Complete State:")
printState3x3(completeState)
print("Left side rotation:")
x = makeMove3x3(completeState, "L", True)

Complete State:
			 Y	Y	Y	
			 Y	Y	Y	
			 Y	Y	Y	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
			 O	O	O	
			 O	O	O	
			 O	O	O	
-------------------------------
Left side rotation:
			 B	Y	Y	
			 B	Y	Y	
			 B	Y	Y	
R	R	R	 O	B	B	 W	W	W	 Y	G	G	
R	R	R	 O	B	B	 W	W	W	 Y	G	G	
R	R	R	 O	B	B	 W	W	W	 Y	G	G	
			 G	O	O	
			 G	O	O	
			 G	O	O	
-------------------------------


Here you can see we moved the left side of the cube, which pivots the Blue front to the top, the Orange bottom to the front, Green back to the bottom, and Yellow top to the back.  If we do this two more times, it should be back to the start state:

In [5]:
for i in range(3):
    makeMove3x3(completeState,"L",True)


			 O	Y	Y	
			 O	Y	Y	
			 O	Y	Y	
R	R	R	 G	B	B	 W	W	W	 B	G	G	
R	R	R	 G	B	B	 W	W	W	 B	G	G	
R	R	R	 G	B	B	 W	W	W	 B	G	G	
			 Y	O	O	
			 Y	O	O	
			 Y	O	O	
-------------------------------
			 G	Y	Y	
			 G	Y	Y	
			 G	Y	Y	
R	R	R	 Y	B	B	 W	W	W	 O	G	G	
R	R	R	 Y	B	B	 W	W	W	 O	G	G	
R	R	R	 Y	B	B	 W	W	W	 O	G	G	
			 B	O	O	
			 B	O	O	
			 B	O	O	
-------------------------------
			 Y	Y	Y	
			 Y	Y	Y	
			 Y	Y	Y	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
			 O	O	O	
			 O	O	O	
			 O	O	O	
-------------------------------


In addition, performing a move, and then it's prime should return to the original state:

In [6]:
completeState = [[["Red", "Red", "Red"],["Red","Red", "Red"],["Red","Red", "Red"]],[["Blue", "Blue","Blue"],["Blue", "Blue","Blue"],["Blue", "Blue","Blue"]],[["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"]],[["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"]],[["White", "White", "White"],["White", "White", "White"],["White", "White", "White"]],[["Green", "Green", "Green"],["Green", "Green", "Green"],["Green", "Green", "Green"]]]
printState3x3(completeState)
_ = makeMove3x3(completeState,"R",True)
_ = makeMove3x3(completeState,"Rprime", True)

			 Y	Y	Y	
			 Y	Y	Y	
			 Y	Y	Y	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
			 O	O	O	
			 O	O	O	
			 O	O	O	
-------------------------------
			 Y	Y	B	
			 Y	Y	B	
			 Y	Y	B	
R	R	R	 B	B	O	 W	W	W	 G	G	Y	
R	R	R	 B	B	O	 W	W	W	 G	G	Y	
R	R	R	 B	B	O	 W	W	W	 G	G	Y	
			 O	O	G	
			 O	O	G	
			 O	O	G	
-------------------------------
			 Y	Y	Y	
			 Y	Y	Y	
			 Y	Y	Y	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
			 O	O	O	
			 O	O	O	
			 O	O	O	
-------------------------------


Let's make sure the new `winner3x3` function works:

In [7]:
completeState = [[["Red", "Red", "Red"],["Red","Red", "Red"],["Red","Red", "Red"]],[["Blue", "Blue","Blue"],["Blue", "Blue","Blue"],["Blue", "Blue","Blue"]],[["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"]],[["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"]],[["White", "White", "White"],["White", "White", "White"],["White", "White", "White"]],[["Green", "Green", "Green"],["Green", "Green", "Green"],["Green", "Green", "Green"]]]
print(winner3x3(completeState))
makeMove3x3(completeState,'R',True)
print(winner3x3(completeState))
makeMove3x3(completeState,'Rprime',True)
print(winner3x3(completeState))

True
			 Y	Y	B	
			 Y	Y	B	
			 Y	Y	B	
R	R	R	 B	B	O	 W	W	W	 G	G	Y	
R	R	R	 B	B	O	 W	W	W	 G	G	Y	
R	R	R	 B	B	O	 W	W	W	 G	G	Y	
			 O	O	G	
			 O	O	G	
			 O	O	G	
-------------------------------
False
			 Y	Y	Y	
			 Y	Y	Y	
			 Y	Y	Y	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
			 O	O	O	
			 O	O	O	
			 O	O	O	
-------------------------------
True


Let's see how our `getTuple3x3` works:

In [8]:
completeState = [[["Red", "Red", "Red"],["Red","Red", "Red"],["Red","Red", "Red"]],[["Blue", "Blue","Blue"],["Blue", "Blue","Blue"],["Blue", "Blue","Blue"]],[["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"]],[["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"]],[["White", "White", "White"],["White", "White", "White"],["White", "White", "White"]],[["Green", "Green", "Green"],["Green", "Green", "Green"],["Green", "Green", "Green"]]]
getTuple3x3(completeState,'R')

((('Red', 'Red', 'Red', 'Red', 'Red', 'Red', 'Red', 'Red', 'Red'),
  ('Blue', 'Blue', 'Blue', 'Blue', 'Blue', 'Blue', 'Blue', 'Blue', 'Blue'),
  ('Yellow',
   'Yellow',
   'Yellow',
   'Yellow',
   'Yellow',
   'Yellow',
   'Yellow',
   'Yellow',
   'Yellow'),
  ('Orange',
   'Orange',
   'Orange',
   'Orange',
   'Orange',
   'Orange',
   'Orange',
   'Orange',
   'Orange'),
  ('White',
   'White',
   'White',
   'White',
   'White',
   'White',
   'White',
   'White',
   'White'),
  ('Green',
   'Green',
   'Green',
   'Green',
   'Green',
   'Green',
   'Green',
   'Green',
   'Green')),
 'R')

Even uglier than before, yet still a valid Tuple.  Let's test it!

## Results

First, let's try just one cube rotation and one repetition.  This took a minute, and didn't finda path, but created a Q with 391,903 values in it.  I did successfully do one run with 10 repetitions that did return a valid path, but I haven't been able to replicate it.

In [10]:
completeState = [[["Red", "Red", "Red"],["Red","Red", "Red"],["Red","Red", "Red"]],[["Blue", "Blue","Blue"],["Blue", "Blue","Blue"],["Blue", "Blue","Blue"]],[["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"]],[["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"]],[["White", "White", "White"],["White", "White", "White"],["White", "White", "White"]],[["Green", "Green", "Green"],["Green", "Green", "Green"],["Green", "Green", "Green"]]]
newstate = completeState
for i in range(1):
    move = random.choice(validMoves(newstate))
    print("Move ", i, " was: ", move)
    newstate = makeMove3x3(newstate,move)


printState3x3(newstate)

startTime = time.time()
Q, steps = trainQ(newstate, 1, 0.5, 0.7, validMoves, makeMove3x3, winner3x3, getTuple3x3)
endTime = time.time()
print(steps)
path, moveList = testQ(Q, newstate, 20000, validMoves, makeMove3x3, winner3x3, getTuple3x3)

print("Training took: ", endTime-startTime, " seconds.")
print("Mean of solution length: ", np.mean(steps))
print("Median of solution length: ", np.median(steps))
print("Q length:", len(Q))
if path == "No path found":
    print(path)
else:
    for i in range(len(path)):
        printState3x3(path[i])
        print("Move: ", moveList[i])


Move  0  was:  Dprime
			 Y	Y	Y	
			 Y	Y	Y	
			 Y	Y	Y	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
R	R	R	 B	B	B	 W	W	W	 G	G	G	
G	G	G	 R	R	R	 B	B	B	 W	W	W	
			 O	O	O	
			 O	O	O	
			 O	O	O	
-------------------------------
[399857]
Training took:  69.70434999465942  seconds.
Mean of solution length:  399857.0
Median of solution length:  399857.0
Q length: 391903
No path found


The time to complete takes a rapdily increasing amount of time. 

In [2]:
completeState = [[["Red", "Red", "Red"],["Red","Red", "Red"],["Red","Red", "Red"]],[["Blue", "Blue","Blue"],["Blue", "Blue","Blue"],["Blue", "Blue","Blue"]],[["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"],["Yellow", "Yellow", "Yellow"]],[["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"],["Orange", "Orange", "Orange"]],[["White", "White", "White"],["White", "White", "White"],["White", "White", "White"]],[["Green", "Green", "Green"],["Green", "Green", "Green"],["Green", "Green", "Green"]]]
newstate = completeState
for i in range(1):
    move = random.choice(validMoves(newstate))
    print("Move ", i, " was: ", move)
    newstate = makeMove3x3(newstate,move)


printState3x3(newstate)

startTime = time.time()
Q, steps = trainQ(newstate, 5, 0.5, 0.7, validMoves, makeMove3x3, winner3x3, getTuple3x3)
endTime = time.time()
print(steps)
path, moveList = testQ(Q, newstate, 20000, validMoves, makeMove3x3, winner3x3, getTuple3x3)

print("Training took: ", endTime-startTime, " seconds.")
print("Mean of solution length: ", np.mean(steps))
print("Median of solution length: ", np.median(steps))
print("Q length:", len(Q))
if path == "No path found":
    print(path)
else:
    for i in range(len(path)):
        printState3x3(path[i])
        print("Move: ", moveList[i])

Move  0  was:  R
			 Y	Y	B	
			 Y	Y	B	
			 Y	Y	B	
R	R	R	 B	B	O	 W	W	W	 G	G	Y	
R	R	R	 B	B	O	 W	W	W	 G	G	Y	
R	R	R	 B	B	O	 W	W	W	 G	G	Y	
			 O	O	G	
			 O	O	G	
			 O	O	G	
-------------------------------


KeyboardInterrupt: 

This took over an hour and didn't complete.  

## Conclusions

Reinforcement Learning doesn't seem to be an effective method for a 3x3 Rubik's Cube.  As I described in the proposal, there are [43 quintillion](http://b.chrishunt.co/how-many-positions-on-a-rubiks-cube) positions for a 3x3 cube.  Multiply that by 8 different moves, results in 344 quintillion possible entries in Q.  This would result in too many states to reside in memory, and also probably too much to reside on a consumer level hard drive.  

I ran this on an 8 core Intel i7 with 16GB RAM.  Making this multi-threading could be one approach to speeding up the process, as it was only using one thread.  [This implies](https://stackoverflow.com/questions/1312331/using-a-global-dictionary-with-threads-in-python) that the trainQ could be rewritten to use a thread pool not to exceed your number of processes.  It appears that the Q dictionary could be global, and as long as it was locked when it was being accessed, theoretically the threads should be able to do their thing.  

The Q dictionary is the item taking up all of the memory, so multithreading wouldn't necessarily use up more RAM, it would just cause it to be filled up faster.  This could have a great speed effect on the 2x2 cube, but I think it would just fill up the system RAM faster for the 3x3.

All told, while I was disappointed that the 3x3 wasn't successful, I'm very pleased with the 2x2 results, and I'd like to hand-enter a starting state based on a randomized real-world cube to ensure that the solution works.